In [4]:
pip install pyspark

     |████████████████████████████████| 212.4 MB 63 kB/s 
     |████████████████████████████████| 198 kB 50.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=86674e5b3f0faec445a17328f4a2501ef4c4c09361f855eb30e96073c7b77b1d
  Stored in directory: /root/.cache/pip/wheels/a5/0a/c1/9561f6fecb759579a7d863dcd846daaa95f598744e71b02c77
Successfully built pyspark


In [28]:
def clean(word):
    word = word.encode("ascii", "ignore").decode()
    regex = re.compile('[%s]' % re.escape(string.punctuation))
    word = regex.sub('', word)
    word = word.strip('"\`”’“')
    return word

In [29]:
def parseUrl(line):
    global category
    category = ['/'.join(line.split()[1].split('/')[6:][:-1])]

In [33]:
def removeStopWords(tokens, stop_words):
    filteredWords = []
    for w in tokens:
        cleanedWord = clean(w.lower())
        if cleanedWord not in stop_words:
            filteredWords.append(cleanedWord)
    return [''.join(category) + '#' + w for w in filteredWords if w not in '']

In [31]:
def parseLine(line, stop_words):
    if 'URL: http://www.nytimes.com/' in line:
        parseUrl(line)
        return []
    return removeStopWords(word_tokenize(line), stop_words)

In [32]:
stop_words = set(stopwords.words('english')) 
text_file = sc.textFile("nytimes_news_articles.txt")
counts = text_file.flatMap(lambda line:parseLine(line, stop_words))
counts1=counts.map(lambda word: (word, 1))
counts2=counts1.reduceByKey(lambda a, b: a + b, 1)
counts3=counts2.map(lambda x: (x[1],x[0])).sortByKey(0, 1)
counts4=counts3.map(lambda x: (x[1],x[0]))
print(counts4.take(10))

[('us/politics#trump', 5583), ('us/politics#clinton', 2895), ('nyregion#new', 2313), ('us/politics#mrs', 2036), ('us/politics#would', 2031), ('nyregion#city', 1759), ('us/politics#campaign', 1709), ('us/politics#republican', 1676), ('nyregion#ms', 1508), ('business#percent', 1483)]
